# Initial Imports & Setup

In [15]:
import pandas as pd
import numpy as np
import glob as glob
import shutil
import os
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
import seaborn as sns
import datetime as datetime

# %matplotlib inline
plt.style.use('fivethirtyeight')
plt.rcParams["svg.fonttype"] = "none"
plt.rcParams["font.family"] = "Bitstream Vera Sans"
plt.rcParams["figure.titleweight"] = 'Bold'
plt.rcParams['figure.titlesize'] = 'xx-large'

# Importing Data into Dataframe

Still working on whether to index or not. Maybe all columns should be data for now. 

In [16]:
names = pd.read_csv('./files/names.csv', index_col='MeridiosName')
metrics = pd.read_csv('./files/metrics.csv', dtype={'MeridiosMetric':object, 'Metric':object})
metrics.set_index('MeridiosMetric', inplace=True)

df = pd.DataFrame()

files = glob.glob('./data/*.csv')
for file in files:
    filedf = pd.read_csv(file,
                        usecols=["NAME","Metricname","SeenNum","SeenDenom"])
    # Prettier Names
    filedf = filedf.rename(columns={'NAME': 'MeridiosName',
                                    'Metricname': 'MeridiosMetric',
                                    'SeenNum': 'Numerator', 
                                    'SeenDenom': 'Denominator'})

    filedf['Name'] = filedf.MeridiosName.map(names.Name)
    filedf['Metric'] = filedf.MeridiosMetric.map(metrics.Metric)
    
    filename_parts = file[7:-4].split(' ')
    # 0 Padded Dates with Dots. 03.15.2018
    if (len(str(filename_parts[0])) is 10):
        filedf['Date'] = datetime.datetime.strptime(filename_parts[0], "%m.%d.%Y")
        filedf['Percentage'] = filedf['Numerator'] / filedf['Denominator'] * 100
        #Not sure how to handle data vs index
        #filedf.set_index(['MeridiosName', 'Metric', 'Date'], inplace=True)
        df = df.append(filedf) 
    else:
        print("CSV Data File doesn't have Zero-Padded Date at beginning.")
#df.dtypes
#df.head()
#df.info()
#df.describe()
#df.columns
#df.Metric.unique()

## Lookups:

* what clinic does a provider belong to? 
* What's a target percentage for a measure

## Cleanups:

* Percentage can be rounded to 2 decimal points (but why throw out the data on the import?)


## Validations:

* Does Data include new MeridiosName, New Metric items? (Has the underlying report changed?)



In [17]:
#for name in df.Name.unique():
name = 'NCFP'

foldername = name.replace(" ", "_")

Provider = df[df['Name'] == name]
Provider.dropna(inplace=True)

for number, metric in enumerate(Provider.Metric.unique()):
    ProviderMetric = Provider[Provider['Metric'] == metric]
    ax = ProviderMetric.plot(x="Date", y="Percentage", legend=False, ylim=(0,100), figsize=(6,6))
    ax.set_xlabel("")

    # Let's Do ever third tick mark (quarters)
    ax.set_xticks(ax.get_xticks()[::3])
    fig = ax.get_figure()
    fig.patch.set_facecolor('none')
    ax.patch.set_facecolor('white')
    ax.patch.set_alpha(0.8)
    fig.suptitle(metric, fontsize=20, fontweight='bold')
    fig.tight_layout(pad=2)

    metricfilename = metric.replace(" ", "_")
    if not os.path.exists("./output/" + foldername):
        os.makedirs("./output/" + foldername)
    fig.savefig("./output/" + foldername + "/" + metricfilename + ".svg", facecolor=fig.get_facecolor(), edgecolor='none')
    plt.close(fig)

shutil.copyfile('./files/index.html', './output/' + foldername + '/index.html') 


/Users/jonathan/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


AttributeError: 'float' object has no attribute 'replace'